# teste de modelo com arvóre de decisão

In [392]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [393]:
df = pd.read_csv(r'/home/lipe/projetos/case-Fm2c/data/TurnOver_Analise.csv', sep=';')

In [394]:
'''
Coluna Situação = variável alvo
Realizar remapeamento dos valores da coluna, Ativos = 1 e Desligados = 2
'''

df.Situação = df.Situação.map({'Ativos': 1, 'Desligados': 0})

In [395]:
'''
Colunas Vale(Alimentação, refeição, transporte)
Colunas Qtd(Dependentes, promoções)
Colunas Conv(Médico, Odonto)

Serão tranformados em inteiros apenas a coluna Qtd de dependentes vai representar a quantidade real de dependentes do colaborador
As outras os valores 1 correspondem a Tem aquele benefício e 0 como Não tem o benefício
'''

df['Tempo de Empresa'] = df['Tempo de Empresa'].str.replace(',','.')
df['Vale Aliment.'] = df['Vale Aliment.'].str.replace(',','.')
df['Vale Refei.'] = df['Vale Refei.'].str.replace(',','.')
df['Vale Transp.'] = df['Vale Transp.'].str.replace(',','.')
df['Qtd. Dependentes'] = df['Qtd. Dependentes'].str.replace(',','.')
df['Conv. Médico'] = df['Conv. Médico'].str.replace(',','.')
df['Conv. Odonto'] = df['Conv. Odonto'].str.replace(',','.')
df['Qtd. Promoções'] = df['Qtd. Promoções'].str.replace(',','.')

df['Tempo de Empresa'] = df['Tempo de Empresa'].astype(float)
df['Vale Aliment.'] = df['Vale Aliment.'].astype(float)
df['Vale Refei.'] = df['Vale Refei.'].astype(float)
df['Vale Transp.'] = df['Vale Transp.'].astype(float)
df['Qtd. Dependentes'] = df['Qtd. Dependentes'].astype(float)
df['Conv. Médico'] = df['Conv. Médico'].astype(float)
df['Conv. Odonto'] = df['Conv. Odonto'].astype(float)
df['Qtd. Promoções'] = df['Qtd. Promoções'].astype(float)

df['Tempo de Empresa'] = df['Tempo de Empresa'].astype(int)
df['Vale Aliment.'] = df['Vale Aliment.'].astype(int)
df['Vale Refei.'] = df['Vale Refei.'].astype(int)
df['Vale Transp.'] = df['Vale Transp.'].astype(int)
df['Qtd. Dependentes'] = df['Qtd. Dependentes'].astype(int)
df['Conv. Médico'] = df['Conv. Médico'].astype(int)
df['Conv. Odonto'] = df['Conv. Odonto'].astype(int)
df['Qtd. Promoções'] = df['Qtd. Promoções'].astype(int)

In [396]:
''' 
tratamento de colunas que possume dados em formato string, limitados até três tipos de dados diferentes
Colunas Grupo empresa e classif Salário
'''

df['GRUPO EMPRESA'] = df['GRUPO EMPRESA'].map({'EMPRESA A': 0, 'EMPRESA B': 1, 'EMPRESA C': 2})
df['Classif. Salário'] = df['Classif. Salário'].map({'Baixo': 0, 'Médio': 1, 'Alto': 2})


In [397]:

'''
Transformar o restante das colunas que tem mais de 3 tipos de dados (Cargo, Estado)
Verificar a existência de dados nulos anso de aplicar o encoder
'''
encoder = LabelEncoder()

df['Matricula'] = encoder.fit_transform(df['Matricula'])
df['Cargo'] = encoder.fit_transform(df['Cargo'])
df['Estado'] = encoder.fit_transform(df['Estado'])


In [398]:
'''
Separar variável alvo das categóricas
Separa conjunto de dados para treinamento e teste
Definir semente para controlar a variação randomica dos dados entre a separação de treino e teste
'''
X_test_colaboradores_ativos = df.query('Situação == 1')
X_test_colaboradores_desligados = df.query('Situação == 0')
X = df[['Cargo', 'GRUPO EMPRESA', 'Matricula', 'Idade', 'Estado', 'Classif. Salário', 'Tempo de Empresa', 'Vale Aliment.', 'Vale Refei.', 'Vale Transp.', 'Qtd. Dependentes', 'Conv. Médico', 'Conv. Odonto', 'Qtd. Promoções']]
y = df.Situação

np.random.seed(5)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
#X_test_colaboradores_ativos.to_csv('data/Dataframe_teste_colaboradores_ativos.csv', index=False)
#X_test_colaboradores_Desligados.to_csv('data/Dataframe_teste_colaboradores_desligados.csv', index=False)

In [400]:
cls  = DecisionTreeClassifier()
cls.fit(X=X_train, y=y_train)

DecisionTreeClassifier()

In [401]:
#preditc = cls.predict(X_test)

#preditc

In [402]:
X_test_colaboradores_ativos.drop('Situação', axis=1, inplace=True)
preditc_colaboradores_ativos = cls.predict(X_test_colaboradores_ativos)

print(preditc_colaboradores_ativos)

[1 1 1 ... 1 1 1]


/tmp/ipykernel_35387/3645090821.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_colaboradores_ativos.drop('Situação', axis=1, inplace=True)


In [403]:
X_test_colaboradores_desligados.drop('Situação', axis=1, inplace=True)
preditc_colaboradores_desligados = cls.predict(X_test_colaboradores_desligados)

preditc_colaboradores_desligados

/tmp/ipykernel_35387/3245029867.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_colaboradores_desligados.drop('Situação', axis=1, inplace=True)


array([0, 0, 0, ..., 0, 0, 0])

In [404]:
print(f'Acurácia: {cls.score(X_test, y_test): .2%}')

Acurácia:  89.71%


In [405]:
'''
Dataframe X_test_colaboradores_ativos Sem aplicar o LabelEncoder com todas as informações dos funcionário ativos
Dataframe X_test_colaboradores_desligados Sem aplicar o LabelEncoder com todas as informações dos funcionário desligados
'''
df_comparativo_ativos = pd.read_csv('Dataframe_teste_colaboradores_ativos.csv', sep=',')
df_comparativo_desligados = pd.read_csv('Dataframe_teste_colaboradores_desligados.csv', sep=',')


In [406]:
df_comparativo_ativos['Predição'] = preditc_colaboradores_ativos
df_comparativo_desligados['Predição'] = preditc_colaboradores_desligados

In [407]:
df_comparativo_ativos.Predição = df_comparativo_ativos.Predição.map({0:'Será desligado', 1:'Não sera desligado'})
df_comparativo_desligados.Predição = df_comparativo_desligados.Predição.map({0:'Será desligado', 1:'Não sera desligado'})

In [ ]:
df_comparativo_ativos.to_csv('data/Análise_predição_ativos.csv', index=False)
df_comparativo_desligados.to_csv('data/Análise_predição_desligados.csv', index=False)